In [68]:
#lee logs de ingestas 

In [84]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
import parametros

In [85]:
#FECHA ACTUAL 

In [86]:
now = datetime.now()
ahora = str(now.strftime("%Y-%m-%dT%H:%M:%S"))

In [87]:
ahora

'2021-09-22T16:47:24'

In [88]:
#conecta elasticsearch

In [89]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

In [90]:
#Leyendo archivo log sonicwall

In [91]:
 try:
    #logread = pd.read_csv('../archivos/sonicwall-curado.log', sep='\t', encoding = 'utf-8')
    header_list = ["column1"]
    logread = pd.read_csv('../archivos/sonicwall-curado.log', header=None,names=header_list, encoding = 'utf-8')
    logread.fillna('', inplace=True)
   
 except:
    print("fecha:",ahora,"- Error leyendo archivo sonicwall-curado.log")
    exit()

In [92]:
logread

column1
0      Fecha maxima en sonic curado: 2021-06-01 10:50...
1      Fecha maxima en indice crudo: 2021-06-21 23:33:00
2      Fecha:  2021-06-21 18:33:02.616155 - Datos CAT...
3      Fecha:  2021-06-21 18:33:02.616155 - Datos DOM...
4      Fecha:  2021-06-21 18:33:02.616155 - Insertado...
...                                                  ...
39244                                        control fr 
39245                                          control 2
39246                                          control 3
39247                                          control 4
39248                                          actualiza

[39249 rows x 1 columns]

In [93]:
list(logread.columns)

['column1']

In [94]:
indice = parametros.monitor_index

In [95]:
use_these_keys = ['column1', '@timestamp']

In [96]:
#Insertando datos en indice

In [83]:
use_these_keys = ['Fecha maxima en sonic curado: 2021-06-01 10:50:00 | Fecha inicio: 2021-06-01 10:50:00','@timestamp']
###Datos que vienen de archivo semilla, hoja DIRECCIONAMIENTO: 'RED INDOOR', 'IP OUTDOOR1','IP OUTDOOR2'
### Datos que vienen de Cambium.device:
###'IP_INDOOR_1', 'IP_OUTDOOR_1', 'IP_OUTDOOR_2', 'MAC_INDOOR_1','MAC_OUTDOOR_1', 'MAC_OUTDOOR_2'
### Estos corresponden a ap_group

def filterKeys(document):
    return {key: document[key] for key in use_these_keys }
timestamp = datetime.now()
# datos_merged['@timestamp'] = timestamp.isoformat()

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": sonuc_wall, 
                "_id": f"{document['Fecha maxima en sonic curado: 2021-06-01 10:50:00 | Fecha inicio: 2021-06-01 10:50:00']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(timestamp))
print("fecha:",ahora,"- Insertados en semilla:", salida[0])

AttributeError: 'datetime.datetime' object has no attribute 'iterrows'